<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/Web_scraping_Accidents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install beautifulsoup4
!pip install mecab-python3

In [0]:
import sys
from datetime import *
from time import *

def get_yobi(inDate):
  
  yobi = ["月","火","水","木","金","土","日"]

  a = datetime.strptime(inDate,'%Y/%m/%d')

  #return yobi[a.weekday()]
  return a.weekday()

In [0]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import pandas as pd
#import MeCab
from google.colab import files

#https://accident.laboneko.jp/accidents?limit=10&offset=0
#https://accident.laboneko.jp/accidents?limit=11680&offset=0

# 親ページURL
url_pr = 'https://accident.laboneko.jp/accidents?limit=11680&offset=0'

# データフレームを定義
columns = ["date","time","yobi","line","from_st","to_st","st"]
df = pd.DataFrame(columns=columns)

html_doc = urlopen(url_pr)

# BeautifulSoupでスクレイピング
soup = BeautifulSoup(html_doc, 'html.parser')
# ラッピング単位のクラスを取得
table  = soup.find_all("table", {"class": "table table-striped"})[0]
rows = table.findAll("tr")

for row in rows:
  dataRow = []
  for cell in row.findAll("td"):
    dataRow.append(cell.text)

  #listに何か入っている場合
  if dataRow:

    v_date = dataRow[0][0:10]
    v_time = dataRow[0][11:16]
    v_yobi = get_yobi(v_date)

    v_line = dataRow[1]

    v_st_info = dataRow[2]
    v_st_info_sp = v_st_info.split("〜")

    if len(v_st_info_sp) >= 2:
      v_from_st = v_st_info_sp[0]
      v_to_st = v_st_info_sp[1]
      v_st = v_st_info_sp[0]
    else:
      v_from_st = v_st_info
      v_to_st = v_st_info
      v_st = v_st_info

    # 各データをデータフレームに格納
    se = pd.Series([v_date,v_time,v_yobi,v_line,v_from_st,v_to_st,v_st], columns)
    df = df.append(se, columns)

# 収集したデータをCSV形式で保存
filename = "Accident_list.csv"
df.to_csv(filename, encoding = 'utf-8')
#files.download(filename)


In [0]:
import seaborn as sns

#index_col=0を付けないとUnnamedというカラムが作成されてしまう。
df_ac = pd.read_csv("Accident_list.csv",index_col=0)
# pdp.ProfileReport(df_ac)

df_ac["line"].value_counts().head(50)

In [97]:
import itertools

kanto_line = pd.read_table('kanto_line.txt', header=None)

#dataframeをlist化
kanto_line_list = kanto_line.values.tolist()
#2次元listから1次元listへ
kanto_line_list = list(itertools.chain.from_iterable(kanto_line_list))

print(kanto_line_list)

['東武東上線', '京浜東北線', '東海道線_(JR東海)', '中央快速線', '常磐線', '小田急小田原線', '西武新宿線', '宇都宮線', '京王線', '京急本線', '西武池袋線', '東武伊勢崎線', '高崎線', '東海道線_(JR東日本)', '東急田園都市線', '山手線', '東北本線', '京成本線', '横浜線', '総武快速線', '横須賀線', '南武線', '埼京線', '相鉄本線', '東武野田線']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [249]:
#関東(首都圏)に影響を与えそうな路線のデータのみ抽出
new_data = df_ac[df_ac['line'].isin(kanto_line_list)]
new_data.head(10)

,date,time,yobi,line,from_st,to_st,st
1,2019/08/07,18:00,2,宇都宮線,東大宮駅,蓮田駅,東大宮駅
9,2019/08/03,23:32,5,小田急小田原線,新松田駅,開成駅,新松田駅
15,2019/08/01,15:23,3,総武快速線,稲毛駅,稲毛駅,稲毛駅
17,2019/07/31,18:09,2,宇都宮線,赤羽駅,赤羽駅,赤羽駅
19,2019/07/31,09:52,2,中央快速線,武蔵小金井駅,武蔵小金井駅,武蔵小金井駅
23,2019/07/30,07:02,1,小田急小田原線,生田駅,生田駅,生田駅
28,2019/07/28,15:03,6,西武池袋線,入間市駅,仏子駅,入間市駅
32,2019/07/27,12:49,5,東武東上線,高坂駅,高坂駅,高坂駅
38,2019/07/25,16:06,3,小田急小田原線,相武台前駅,相武台前駅,相武台前駅
41,2019/07/22,21:19,0,常磐線,柏駅,柏駅,柏駅


In [0]:
!pip install pandas-profiling

In [103]:
import pandas_profiling as pdp
from IPython.display import HTML

profile = pdp.ProfileReport(new_data)
profile.to_file(outputfile="myOutputFile_line.html")

#html化して表示
HTML(filename='myOutputFile_line.html')

Number of variables,8
Number of observations,3926
Total Missing (%),0.0%
Total size in memory,245.5 KiB
Average record size in memory,64.0 B
Numeric,2
Categorical,6
Boolean,0
Date,0
Text (Unique),0
Rejected,0


# **profilingで分かったこと**
・曜日は関係無さそう

・分析する上で情報が不足している

・正規化、細分化できる情報は無いか

In [259]:
# データフレームを定義
#通勤時間帯/帰宅時間帯/日中/それ以外で分けてみよう

#コピーを付けないとコピー元も変更されてしまう
new_df = new_data.copy()

#カラム「time」をコピーして追加
#(後でカテゴリ分けしたい為)
new_df = new_df.assign(time_category=new_df['time'])
new_df

#数値型で比較したい場合は「.str[0:2].astype(int)」を使用する
new_df.loc[(new_df['time_category'].str[0:2].isin(["04","05"])) ,"time_category"] = "morning_time"
new_df.loc[(new_df['time_category'].str[0:2].isin(["06","07","08","09"])) ,"time_category"] = "comp_time"
new_df.loc[(new_df['time_category'].str[0:2].isin(["17","18","19","20","21"])) ,"time_category"] = "home_time"
new_df.loc[(new_df['time_category'].str[0:2].isin(["22","23","00","01"])) ,"time_category"] = "night_time"
new_df.loc[(new_df['time_category'].str[0:2].isin(["02","03","10","11","12","13","14","15","16"])) ,"time_category"] = "etc_time"

dmy_df = pd.get_dummies(new_df['time_category'])
time_add_df = pd.merge(new_df, dmy_df, left_index=True, right_index=True)

time_add_df.head(20)

,date,time,yobi,line,from_st,to_st,st,time_category,comp_time,etc_time,home_time,morning_time,night_time
1,2019/08/07,18:00,2,宇都宮線,東大宮駅,蓮田駅,東大宮駅,home_time,0,0,1,0,0
9,2019/08/03,23:32,5,小田急小田原線,新松田駅,開成駅,新松田駅,night_time,0,0,0,0,1
15,2019/08/01,15:23,3,総武快速線,稲毛駅,稲毛駅,稲毛駅,etc_time,0,1,0,0,0
17,2019/07/31,18:09,2,宇都宮線,赤羽駅,赤羽駅,赤羽駅,home_time,0,0,1,0,0
19,2019/07/31,09:52,2,中央快速線,武蔵小金井駅,武蔵小金井駅,武蔵小金井駅,comp_time,1,0,0,0,0
23,2019/07/30,07:02,1,小田急小田原線,生田駅,生田駅,生田駅,comp_time,1,0,0,0,0
28,2019/07/28,15:03,6,西武池袋線,入間市駅,仏子駅,入間市駅,etc_time,0,1,0,0,0
32,2019/07/27,12:49,5,東武東上線,高坂駅,高坂駅,高坂駅,etc_time,0,1,0,0,0
38,2019/07/25,16:06,3,小田急小田原線,相武台前駅,相武台前駅,相武台前駅,etc_time,0,1,0,0,0
41,2019/07/22,21:19,0,常磐線,柏駅,柏駅,柏駅,home_time,0,0,1,0,0


In [262]:
profile2 = pdp.ProfileReport(time_add_df)
profile2.to_file(outputfile="myOutputFile_line2.html")

#html化して表示
HTML(filename='myOutputFile_line2.html')

Number of variables,14
Number of observations,3926
Total Missing (%),0.0%
Total size in memory,295.3 KiB
Average record size in memory,77.0 B
Numeric,2
Categorical,7
Boolean,5
Date,0
Text (Unique),0
Rejected,0
